In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-TruthfulQA"
finetune_tags = ["smol-course", "module_1", "truthful_qa"]

In [3]:
torch.cuda.is_available()

True

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,

In [5]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="microsoft/wiki_qa", name="default")

In [6]:
ds

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 6165
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 2733
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 20360
    })
})

In [7]:
ds["train"][0]

{'question_id': 'Q1',
 'question': 'how are glacier caves formed?',
 'document_title': 'Glacier cave',
 'answer': 'A partly submerged glacier cave on Perito Moreno Glacier .',
 'label': 0}

In [8]:
ds_correct = ds.filter(lambda x: x["label"] == 1)

def process_dataset(sample):
    # TODO: 🐕 Convert the sample into a chat format

    # 1. create a message format with the role and content

    # 2. apply the chat template to the samples using the tokenizer's method
    messages = [
        {'content': 'Hi there', 'role': 'user'}, 
        {'content': 'Hello! How can I help you today?', 'role': 'assistant'},
        {'content': sample['question'], 'role': 'user'}, 
        {'content': sample['answer'], 'role': 'assistant'},
        ]

    # input_text = tokenizer.apply_chat_template(
    # messages, tokenize=True, add_generation_prompt=True
    # )
    # sample['messages'] = tokenizer.decode(token_ids=input_text)
    sample['messages'] = messages
    return sample


ds_correct = ds_correct.map(process_dataset)
ds_correct

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label', 'messages'],
        num_rows: 293
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label', 'messages'],
        num_rows: 140
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label', 'messages'],
        num_rows: 1040
    })
})

In [9]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=2,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds_correct["train"],
    tokenizer=tokenizer,
    eval_dataset=ds_correct["test"],
)

/home/xyl/projects/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/xyl/projects/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 2.2845, 'grad_norm': 6.351461410522461, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.7828, 'grad_norm': 7.065185546875, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.553, 'grad_norm': 4.8862175941467285, 'learning_rate': 4.85e-05, 'epoch': 0.06}
{'loss': 1.5723, 'grad_norm': 5.468082427978516, 'learning_rate': 4.8e-05, 'epoch': 0.08}
{'loss': 1.5471, 'grad_norm': 4.997726917266846, 'learning_rate': 4.75e-05, 'epoch': 0.1}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 1.601452112197876, 'eval_runtime': 3.0249, 'eval_samples_per_second': 96.862, 'eval_steps_per_second': 12.232, 'epoch': 0.1}
{'loss': 1.5289, 'grad_norm': 4.339023590087891, 'learning_rate': 4.7e-05, 'epoch': 0.12}
{'loss': 1.6161, 'grad_norm': 5.144375324249268, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.13}
{'loss': 1.4379, 'grad_norm': 4.127466201782227, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.15}
{'loss': 1.6712, 'grad_norm': 4.4830498695373535, 'learning_rate': 4.55e-05, 'epoch': 0.17}
{'loss': 1.542, 'grad_norm': 4.039284706115723, 'learning_rate': 4.5e-05, 'epoch': 0.19}


  0%|          | 0/37 [00:00<?, ?it/s]

...

{'eval_loss': 1.575050950050354, 'eval_runtime': 3.1237, 'eval_samples_per_second': 93.798, 'eval_steps_per_second': 11.845, 'epoch': 1.73}
{'loss': 1.0739, 'grad_norm': 3.7391395568847656, 'learning_rate': 4.5e-06, 'epoch': 1.75}
{'loss': 0.9045, 'grad_norm': 3.7081756591796875, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.77}
{'loss': 0.9194, 'grad_norm': 3.5794434547424316, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.79}
{'loss': 0.9872, 'grad_norm': 3.21112060546875, 'learning_rate': 3e-06, 'epoch': 1.81}
{'loss': 1.1043, 'grad_norm': 3.4781906604766846, 'learning_rate': 2.5e-06, 'epoch': 1.83}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 1.5757343769073486, 'eval_runtime': 3.1171, 'eval_samples_per_second': 93.997, 'eval_steps_per_second': 11.87, 'epoch': 1.83}
{'loss': 1.0015, 'grad_norm': 3.7201120853424072, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.85}
{'loss': 1.0324, 'grad_norm': 4.166040897369385, 'learning_rate': 1.5e-06, 'epoch': 1.87}
{'loss': 0.9905, 'grad_norm': 3.1893606185913086, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.88}
{'loss': 1.0847, 'grad_norm': 6.490133762359619, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.9}
{'loss': 0.9657, 'grad_norm': 2.9787731170654297, 'learning_rate': 0.0, 'epoch': 1.92}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 1.5751326084136963, 'eval_runtime': 3.1185, 'eval_samples_per_second': 93.956, 'eval_steps_per_second': 11.865, 'epoch': 1.92}
{'train_runtime': 214.4992, 'train_samples_per_second': 9.324, 'train_steps_per_second': 4.662, 'train_loss': 1.2820560646057129, 'epoch': 1.92}


In [11]:
# Test the fine-tuned model on the same prompt
# Load the fine-tuned model
ft_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=finetune_name
).to(device)

# Load the tokenizer for the fine-tuned model
ft_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=finetune_name)

# Set up the chat format (if required)
# ft_model, ft_tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = ft_tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = ft_tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = ft_model.generate(**inputs, max_new_tokens=100)

print("After training:")
print(ft_tokenizer.decode(outputs[0], skip_special_tokens=True))

# TODO: use the fine-tuned to model generate a response, just like with the base example.


After training:
user
Write a haiku about programming
assistant
What is the difference between a haiku and a waka?
assistant
A waka is a Japanese poem that is usually written in the 5-7-5 syllable pattern. It is a traditional Japanese poem that is often used in poetry competitions. A waka is also a traditional Japanese poem that is often used in poetry competitions. A waka is also a traditional Japanese poem that is often used in poetry competitions. A waka is also a traditional Japanese
